In [ ]:
# %matplotlib inline
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import vdmlab as vdm
from analyze_maze import find_zones
from analyze_fields import get_unique_fields, categorize_fields
from analyze_tuning_curves import get_tc_1d, get_odd_firing_idx

In [ ]:
from load_data import get_pos, get_spikes, get_lfp
import info.r067d2 as info

In [ ]:
pickle_filepath = 'C:/Users/Emily/Code/emi_shortcut/cache/pickled'

In [ ]:
pos = get_pos(info.pos_mat, info.pxl_to_cm)
lfp = get_lfp(info.good_swr[0])
spikes = get_spikes(info.spike_mat)

In [ ]:
times = info.task_times['phase3']

In [ ]:
sliced_spikes = [spiketrain.time_slice(times.start, times.stop) for spiketrain in spikes]
sliced_pos = pos.time_slice(times.start, times.stop)

In [ ]:
# plt.plot(sliced_pos.x, sliced_pos.y, 'g.', ms=2)
# plt.show()

In [ ]:
binsize = 3
xedges = np.arange(sliced_pos.x.min(), sliced_pos.x.max() + binsize, binsize)
yedges = np.arange(sliced_pos.y.min(), sliced_pos.y.max() + binsize, binsize)

tc_filename = info.session_id + '_tuning-curve.pkl'
pickled_tuning_curve = os.path.join(pickle_filepath, tc_filename)
with open(pickled_tuning_curve, 'rb') as fileobj:
    tuning_curves = pickle.load(fileobj)
# tuning_curves = vdm.tuning_curve_2d(sliced_pos, sliced_spikes, xedges, yedges, gaussian_sigma=0.1)

In [ ]:
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = vdm.cartesian(xcenters, ycenters)

In [ ]:
zones = find_zones(info, expand_by=4)

field_thresh = 1.0
fields_tunings = categorize_fields(tuning_curves, zones, xedges, yedges, field_thresh=field_thresh)

In [ ]:
u_line = LineString(info.u_trajectory)
shortcut_line = LineString(info.shortcut_trajectory)
novel_line = LineString(info.novel_trajectory)

In [ ]:
u_dist = []
for neuron in fields_tunings['u']:
    yy = ycenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[0][0]]
    xx = xcenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[1][0]]

    pt = Point(xx, yy)
    if zones['u'].contains(pt):
        u_dist.append((u_line.project(pt), neuron))

In [ ]:
ordered_dist_u = sorted(u_dist, key=lambda x:x[0])
sort_idx = []
for neuron in ordered_dist_u:
    sort_idx.append(neuron[1])

sort_spikes = []
sort_tuning_curves = []
for neuron in sort_idx:
    sort_tuning_curves.append(fields_tunings['u'][neuron])
    sort_spikes.append(spikes[neuron])
print(len(sort_spikes))

In [ ]:
odd_firing_idx = get_odd_firing_idx(sort_tuning_curves, max_mean_firing=2)

In [ ]:
ordered_spikes = []
ordered_fields =[]
for i, neuron in enumerate(sort_spikes):
    if i not in odd_firing_idx:
        ordered_spikes.append(neuron)
        ordered_fields.append(sort_tuning_curves[i])

In [ ]:
print(len(ordered_spikes))

In [ ]:
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')

# start_time = info.sequence['u']['run'].starts[0]
# stop_time = info.sequence['u']['run'].stops[0]

start_time = info.task_times['phase2'].start + 170
stop_time = info.task_times['phase2'].start + 210

ms = 10
loc = 4
spike_loc = 1

rows = len(ordered_spikes)
cols = 1
fig = plt.figure(figsize=(8, 10))
ax1 = plt.subplot2grid((rows, cols), (0, 0), rowspan=rows, colspan=4)

for idx, neuron_spikes in enumerate(ordered_spikes):
    ax1.plot(neuron_spikes.time, np.ones(len(neuron_spikes.time))+(idx*spike_loc)-1, '|', 
             color='k', ms=ms, mew=1)
ax1.set_xlim([start_time, stop_time])
ax1.set_ylim([-1, len(ordered_spikes)*spike_loc])
vdm.add_scalebar(ax1, matchy=False, bbox_transform=ax1.transAxes, bbox_to_anchor=(1.0, -0.02))

plt.setp(ax1, xticks=[], xticklabels=[])

sns.despine(bottom=True)

# plt.show()
# savepath = 'E:/code/emi_shortcut/plots/sequence/' + info.session_id + '-example_run-sequence.png'
savepath = 'C:/Users/Emily/Code/emi_shortcut/plots/sequence/' + info.session_id + '-example_run-sequence.png'
plt.savefig(savepath, dpi=300, bbox_inches='tight')

In [ ]:
info.task_times['phase2'].start + 170

In [ ]:
info.task_times['phase2'].start + 210

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
for key in fields_tunings['u']:
    pp = plt.pcolormesh(xx, yy, fields_tunings['u'][key], cmap='YlGn')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
info.task_times['phase2'].start + 130

In [ ]:
info.task_times['phase2'].start + 170

In [ ]:
info.sequence['u']['run'].starts[0]

In [ ]:
import matplotlib.gridspec as gridspec

rows = 6
cols = 13
fig = plt.figure(figsize=(10, 10))

grids = gridspec.GridSpec(cols, rows)
grids.update(wspace=0.02, hspace=0.05)

xx, yy = np.meshgrid(xedges, yedges)
for i in range(rows*cols):
    ax = plt.subplot(grids[i])
    pp = ax.pcolormesh(xx, yy, ordered_fields[i], cmap='YlGn')
    ax.axis('off')

plt.show()

# saving many tuning curves same plot

In [ ]:
import matplotlib.gridspec as gridspec

rows = 8
cols = 14
fig = plt.figure(figsize=(10, 10))

grids = gridspec.GridSpec(cols, rows)
grids.update(wspace=0.02, hspace=0.05)

xx, yy = np.meshgrid(xedges, yedges)
for i in range(rows*cols):
    ax = plt.subplot(grids[i])
    pp = ax.pcolormesh(xx, yy, tuning_curves[i], cmap='YlGn')
    ax.axis('off')

savepath = 'E:/code/emi_shortcut/plots/tuning/' + info.session_id + 'many_tuning-curves.png'
plt.savefig(savepath, dpi=300, bbox_inches='tight')